## WiP D213 Task 2: NLP Deep Learning

### Non-narrative steps

 #### 1. Select dataset
 #### 2. Data EDA/Preprocessing
        - Detection and handling of unusual characters (i.e., emojis and non-English characters)
        - Determine vocabulary size
        - Select word embedding length based on statistical inference
 #### 3. `Tokenization`/normalization
 #### 4. `Padding` (standardize `length of sequences`)
        - Pad as prefix or suffix
        - Pull out at least one example of padded sequence for observation
 #### 5. Determine number of `categories of sentiment` to use and `activation function` to use (specifically in the final Dense layer?)
 #### 6. Determine `train/test split` (<font size=2em>*model.fit(validation_set=`x`)*</font>)
 #### 7. Design the model in `TensorFlow`
        - Number of layers
        - Type of layers
        - Total number of parameters
 #### 8. Choice of `hyperparameters` including:
        - Activation function (see 5.)
              - 'relu'
        - Number of nodes per layer
              - TBD
        - Loss function
              - Probably 'categorical_crossentropy', but backup with reference to the literature
        - Optimizer
              - Probably 'adam', but same as above
        - EarlyStopping criteria
              - Maybe 2? Experiment and decide
        - Evaluation metric
              - MSE or RMSE
 #### 9. `Evaluate` the model
        - Find out how changing EarlyStopping(x) impacts the model vs number of training epochs
        - Make a line graph of the model training both loss metric and validation metric (i.e., MSE)
        - Determine model fitness vs overfitting
        - Quantify accuracy of the model